<a href="https://colab.research.google.com/github/paridhika/CoAP_Parking/blob/master/CSC2515_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Active Transfer Learning

In this project we experiment different active learning setups. To run the code with 

## Importing and setting GPUs

In [ ]:
import tensorflow as tf
import torch
import os
import pickle
import pandas as pd
!pip install transformers
!pip install scikeras[tensorflow]
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime
import random
import matplotlib.pyplot as plt
# matplotlib inline
import json
import seaborn as sns
from tensorflow import keras
from sklearn.metrics import matthews_corrcoef
from scikeras.wrappers import KerasClassifier
!pip install modAL

tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [4]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

## Loading and preprocessing data :
We can either use the preprocess data fetched from my Github repo or them in the last section then running these cells

In [5]:
!git clone "https://github.com/parsafarinnia/sentiment_new_approach"


Cloning into 'sentiment_new_approach'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 75 (delta 20), reused 42 (delta 6), pack-reused 0
Unpacking objects: 100% (75/75), 55.86 MiB | 5.65 MiB/s, done.


In [6]:
topic_num=0

all_topics = pickle.load(open("/content/sentiment_new_approach/all_topics_with_meta.p","rb"))
data=pd.DataFrame.from_dict(all_topics[0]).T
data['topic']=0
tags = data.tag.values

for i in range(1,9):
  topic = pd.DataFrame.from_dict(all_topics[i]).T
  topic['topic'] = i
  data = pd.concat([data,topic],axis=0)


#Converting boolean features into integers in the dataset
map={"rumours":1,"non-rumours":0}
data=data.replace({'tag':map})
map={"photo":1,"none":0}
data=data.replace({'media_type':map})
map={True:1,False:0}
data=data.replace({'verified':map})


#Normalizing features
import copy
df=copy.deepcopy(data)
cols=['favorite_count_log','retweet_count','followers','follow_ratio','length','capital_ratio']
for item in cols:
  column=item
  df[column] = (df[column]-df[column].min()) /(df[column].max()-df[column].min())


data=df
train_df = data[data['topic']!=topic_num]
train_sentences=train_df.text.values
train_labels=train_df.tag.values

test_df = data[data['topic']==topic_num]
test_sentences=test_df.text.values
test_labels=test_df.tag.values


data2=data.reset_index()
def pd_iter_func(df,topic):
    for row in df.itertuples():
        # Define your criteria here
        if row.topic==topic:
            return row

start_test=pd_iter_func(data2,topic_num).Index

if topic_num==8: 
  end_test=len(data2)
else:
  end_test=pd_iter_func(data2,topic_num+1).Index-1

train_start=start_test
train_end=end_test

## Active learning multiple runs
since every model might have a different input out put and training specific setting we have multiple functions for different model. The Multi part comes from the fact that we need to run the model multiple time to avoid the randomness effect of selecting a specific train test. we do this because our data set is around 2k and we want conrecete results


###Run_multy_episolon_greedy

In [7]:
from sklearn.metrics import f1_score
from modAL.models import ActiveLearner
from modAL.batch import uncertainty_batch_sampling
from modAL.uncertainty import uncertainty_sampling
from functools import partial
import random
from sklearn.model_selection import train_test_split
from collections import Counter

In [8]:
from sklearn.metrics import confusion_matrix
def run_multy_epsilon_greedy(BATCH_SIZE,data_test,test_labels,num_of_run,model_name,strategy,train_size,is_random):

  test_pool_split=0.5
  preset_batch = partial(strategy, n_instances=40)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(X_pool_0,y_pool_0, test_size=test_pool_split,shuffle=True )
  N_QUERIES = int(len(X_pool_0)/BATCH_SIZE)

  pref_hist_multy_accuracy=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_multy_f1=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_majority = np.zeros((num_of_run,N_QUERIES+1))
  pref_confusion = np.zeros((num_of_run,N_QUERIES+1,4))
  pref_random = 0
  for i in range(num_of_run):

    clf = get_model(model_name)
    
    X_pool, X_train, y_pool, y_train = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True,random_state=random_seed_list[i])
    X_pool, X_test, y_pool, y_test = train_test_split(X_pool,y_pool, test_size=test_pool_split,shuffle=True,random_state=random_seed_list[i])
    
    learner = ActiveLearner(estimator=clf,query_strategy=preset_batch,X_training=X_train, y_training=y_train)
    t1 = time.time()

    #Allow our model to query our unlabeled dataset for the most informative points according to our query strategy (uncertainty sampling).
    counter_random= 1 

    #Calculate initial batch
    y_pred = learner.predict(X_test)
    macro=f1_score(y_test,y_pred, average='macro')
    pref_hist_multy_f1[i][0]=macro
    tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
    pref_confusion[i][0] = [tn, fp, fn, tp]

    #Calculate initial batch majority
    counts = np.bincount(y_train)
    value = np.argmax(counts)
    majority =[value for i in range(len(y_test))]
    macro=f1_score(y_test,majority, average='macro')
    pref_hist_majority[i][0]=macro

    #Calculate random
    y_rand=[random.randint(0,2) for i in range(len(y_test))]
    macro=f1_score( y_test,y_rand, average='macro')
    pref_random = pref_random + macro


    #Active Leraning loop
    for index in range(1,N_QUERIES+1):
      counter_random += 1 
      t2 = time.time()
      print(i,'th run and query number',index)
      if not is_random:
        query_index, query_instance = learner.query(X_pool)
      print('num if query',len(query_index))
      if is_random:
        index_list = range(len(X_pool))
        query_index = random.sample(index_list,BATCH_SIZE)


      X, y = X_pool[query_index], y_pool[query_index]
      learner.teach(X=X, y=y)

      #Remove the queried instance from the unlabeled pool.
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)

      index_list = range(len(X_pool))
      query_index = random.sample(index_list,10)

      X, y = X_pool[query_index], y_pool[query_index]
      learner.teach(X=X, y=y)

      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)

      #Calculate and report our model's accuracy.
      model_accuracy = learner.score(X_test, y_test)
      y_pred = learner.predict(X_test)
      
    
      macro=f1_score( y_test,y_pred, average='macro')
      print('after query {n}: Accuracy :{acc:0.4f} macro f1 :{f1:0.4f}'.format(n=index + 1, acc=model_accuracy,f1=macro))

      #Save our model's performance for plotting.
      tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
      pref_confusion[i][index] = [tn, fp, fn, tp]
      pref_hist_multy_accuracy[i][index]=model_accuracy
      pref_hist_multy_f1[i][index]=macro

      #Calculate and add majority
      counts = np.bincount(y_train)
      value = np.argmax(counts)
      majority =[value for i in range(len(y_test))]
      macro=f1_score(y_test,majority, average='macro')
      pref_hist_majority[i][index]=macro
      print(format_time(time.time()-t2))

    print(format_time(time.time()-t1))
  pref_random= pref_random/num_of_run
  pref_hist_majority_avg=pref_hist_majority.mean(0)
  pref_hist_multy_acc_avg=pref_hist_multy_accuracy.mean(0)
  pref_hist_multy_f1_avg=pref_hist_multy_f1.mean(0)

  return pref_hist_multy_accuracy,pref_hist_multy_f1,pref_random,pref_hist_majority,N_QUERIES 

### Run_multy_sklearn

In [9]:
from sklearn.metrics import f1_score
from keras import utils as np_utils
from modAL.models import ActiveLearner
from modAL.batch import uncertainty_batch_sampling
from functools import partial
import random
random_seed_list=[5,12,42,29,54]
def run_multy_sklearn(BATCH_SIZE,data_test,test_labels,num_of_run,model_name,strategy,train_size,is_random):

  test_pool_split=0.5
  preset_batch = partial(strategy, n_instances=BATCH_SIZE)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True )
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(X_pool_0,y_pool_0, test_size=test_pool_split,shuffle=True )
  N_QUERIES = int(len(X_pool_0)/BATCH_SIZE)

  pref_confusion = np.zeros((num_of_run,N_QUERIES+1,4))
  pref_hist_multy_accuracy=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_multy_f1=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_majority = np.zeros((num_of_run,N_QUERIES+1))
  pref_random = 0
  for i in range(num_of_run):
    clf = get_model(model_name)
    X_pool, X_train, y_pool, y_train = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True,random_state=random_seed_list[i])
    X_pool, X_test, y_pool, y_test = train_test_split(X_pool,y_pool, test_size=test_pool_split,shuffle=True,random_state=random_seed_list[i])
    
    learner = ActiveLearner(estimator=clf,query_strategy=preset_batch,X_training=X_train, y_training=y_train)
    t1 = time.time()

    #Calculate initial batch
    y_pred = learner.predict(X_test)
    macro=f1_score(y_test,y_pred, average='macro')
    pref_hist_multy_f1[i][0]=macro
    tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
    pref_confusion[i][0] = [tn, fp, fn, tp]

    #Calculate initial batch majority
    counts = np.bincount(y_train)
    value = np.argmax(counts)
    majority =[value for i in range(len(y_test))]
    macro=f1_score(y_test,majority, average='macro')
    pref_hist_majority[i][0]=macro

    #Calculate random
    y_rand=[random.randint(0,2) for i in range(len(y_test))]
    macro=f1_score( y_test,y_rand, average='macro')
    pref_random = pref_random + macro

    #Active Leraning loop
    for index in range(1,N_QUERIES+1):
      t2 = time.time()
      print(i,'th run and query number',index)
      query_index, query_instance = learner.query(X_pool)
      print('num if query',len(query_index))
      if is_random : #or counter_random % random_ratio == 0:
        index_list = range(len(X_pool))
        query_index = random.sample(index_list,BATCH_SIZE)

      X, y = X_pool[query_index], y_pool[query_index]

      for j in range(1):
        learner.teach(X=X, y=y)

      #Remove the queried instance from the unlabeled pool.
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)

      #Calculate and report our model's accuracy.
      model_accuracy = learner.score(X_test, y_test)
      y_pred = learner.predict(X_test)

      macro=f1_score( y_test,y_pred, average='macro')
      print('after query {n}: Accuracy :{acc:0.4f} macro f1 :{f1:0.4f}'.format(n=index + 1, acc=model_accuracy,f1=macro))

      #Save our model's performance for plotting
      tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
      pref_confusion[i][index] = [tn, fp, fn, tp]
      pref_hist_multy_accuracy[i][index]=model_accuracy
      pref_hist_multy_f1[i][index]=macro

      #Calculate and add majority
      counts = np.bincount(y_train)
      value = np.argmax(counts)
      majority =[value for i in range(len(y_test))]
      macro=f1_score(y_test,majority, average='macro')
      pref_hist_majority[i][index]=macro
      print(format_time(time.time()-t2))

    print(format_time(time.time()-t1))
  pref_random= pref_random/num_of_run
  pref_hist_majority_avg=pref_hist_majority.mean(0)
  pref_hist_multy_acc_avg=pref_hist_multy_accuracy.mean(0)
  pref_hist_multy_f1_avg=pref_hist_multy_f1.mean(0)

  return pref_hist_multy_accuracy,pref_hist_multy_f1,pref_random,pref_hist_majority,N_QUERIES

### run_multy

In [24]:
from sklearn.metrics import f1_score
from keras import utils as np_utils
from modAL.models import ActiveLearner
from modAL.batch import uncertainty_batch_sampling
from functools import partial
import random
random_seed_list=[5,12,42,29,54]
def run_multy(BATCH_SIZE,data_test,test_labels,num_of_run,model,strategy,train_size,is_random):
  test_pool_split=0.5
  preset_batch = partial(strategy, n_instances=BATCH_SIZE)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(data_test,test_labels, test_size=train_size,shuffle=True )
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(X_pool_0,y_pool_0, test_size=test_pool_split,shuffle=True )
  N_QUERIES = int(len(X_pool_0)/BATCH_SIZE)

  print(num_of_run,N_QUERIES)
  pref_hist_multy_accuracy=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_multy_f1=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_majority = np.zeros((num_of_run,N_QUERIES+1))
  pref_random = 0

  test_labels = keras.utils.np_utils.to_categorical(test_labels, 2)  
  
  for i in range(num_of_run):
    X_pool, X_train, y_pool, y_train = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True,random_state=random_seed_list[i])
    X_pool, X_test, y_pool, y_test = train_test_split(X_pool,y_pool, test_size=test_pool_split,shuffle=True,random_state=random_seed_list[i])
    
    clf = KerasClassifier(build_fn=model, epochs=50, batch_size=35, verbose=0)

    print('x pool length',X_train.shape)
    print('y pool length',y_train.shape)
    print("Paridhika")
    learner = ActiveLearner(
    estimator=clf,
    query_strategy=preset_batch,
    X_training=X_train, y_training=y_train)
    t1 = time.time()
    print('8888888888888888888888888888888888888888')
    print(y_pool[0],y_pool[1],y_pool[2])
    print('shapeè',y_pool.shape)
  
    #Calculate initial batch
    y_pred = learner.predict(X_test)
    y_test_cat = np.argmax(y_test,axis=1)
    macro=f1_score(y_test_cat,y_pred, average='macro')
    pref_hist_multy_f1[i][0]=macro

    #Calculate initial batch majority
    y_train_cat = np.argmax(y_test,axis=1)
    counts = np.bincount(y_train_cat)
    value = np.argmax(counts)
    majority =[value for i in range(len(y_test_cat))]
    macro=f1_score(y_test_cat,majority, average='macro')
    pref_hist_majority[i][0]=macro

    #Calculate random
    y_rand=[random.randint(0,2) for i in range(len(y_test_cat))]
    macro=f1_score( y_test_cat,y_rand, average='macro')
    pref_random = pref_random + macro

    for index in range(1,N_QUERIES+1):

      print('x pool length',len(X_pool))
      print('y pool length',len(y_pool))
      print(i,'th run and query number',index)
      
      query_index, query_instance = learner.query(X_pool)
      print('num if query',len(query_index))

      if is_random:
        index_list = range(len(X_pool))
        query_index = random.sample(index_list,BATCH_SIZE)
    
      X, y = X_pool[query_index], y_pool[query_index]
      for j in range(5):
        learner.teach(X=X, y=y)

      #Remove the queried instance from the unlabeled pool.
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)
      print('sec',y_pool.shape)

      #Calculate and report our model's accuracy.
      model_accuracy = learner.score(data_test, test_labels)
      y_pred = learner.predict(X_test)
      
      #y_pred_cat=np.argmax(y_pred)
      y_test_cat=np.argmax(y_test,axis=1)
      print('shapeè',y_test_cat.shape)
      macro=f1_score( y_test_cat,y_pred, average='macro')
      print('after query {n}: Accuracy :{acc:0.4f} macro f1 :{f1:0.4f}'.format(n=index + 1, acc=model_accuracy,f1=macro))

      #Save our model's performance for plotting
      pref_hist_multy_accuracy[i][index]=model_accuracy
      pref_hist_multy_f1[i][index]=macro
      # performance_history_acc.append(model_accuracy)
      # performance_history_f1.append(macro)
      # perf_hist_multy[i][0].append(model_accuracy)
      # perf_hist_multy[i][1].append(macro)

      #Calculate and add majority
      y_train_cat = np.argmax(y_test,axis=1)
      counts = np.bincount(y_train_cat)
      value = np.argmax(counts)
      majority =[value for i in range(len(y_test_cat))]
      macro=f1_score(y_test_cat,majority, average='macro')
      pref_hist_majority[i][index]=macro
      
    print(format_time(time.time()-t1))
  pref_random= pref_random/num_of_run
  pref_hist_majority_avg=pref_hist_majority.mean(0)
  pref_hist_multy_acc_avg=pref_hist_multy_accuracy.mean(0)
  pref_hist_multy_f1_avg=pref_hist_multy_f1.mean(0)

  return pref_hist_multy_accuracy,pref_hist_multy_f1,pref_random,pref_hist_majority,N_QUERIES
    

### Run_multy (for pytorch MLP)

In [26]:
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score
from modAL.models import ActiveLearner

from modAL.batch import uncertainty_batch_sampling
from functools import partial
import random
random_seed_list=[5,12,42,29,54]
def run_multy(BATCH_SIZE,data_test,test_labels,num_of_run,model,strategy,train_size,is_random):
  test_pool_split=0.5
  preset_batch = partial(strategy, n_instances=BATCH_SIZE)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(data_test,test_labels, test_size=train_size,shuffle=True )
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(X_pool_0,y_pool_0, test_size=test_pool_split,shuffle=True )
  N_QUERIES = int(len(X_pool_0)/BATCH_SIZE)
  print(num_of_run,N_QUERIES)
  pref_hist_multy_accuracy=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_multy_f1=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_majority = np.zeros((num_of_run,N_QUERIES+1))
  pref_random = 0

  test_labels = keras.utils.to_categorical(test_labels, 2)  
  for i in range(num_of_run):
    X_pool, X_train, y_pool, y_train = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True,random_state=random_seed_list[i])
    X_pool, X_test, y_pool, y_test = train_test_split(X_pool,y_pool, test_size=test_pool_split,shuffle=True,random_state=random_seed_list[i])
    clf = KerasClassifier(model=model)
    print('x pool length',X_train.shape)
    print('y pool length',y_train.shape)
    learner = ActiveLearner(
    estimator=clf,
    query_strategy=preset_batch,
    X_training=X_train, y_training=y_train)

    t1 = time.time()

    # Allow our model to query our unlabeled dataset for the most
    # informative points according to our query strategy (uncertainty sampling).
    
    # calculate initial batch
    y_pred = learner.predict(X_test)
    y_test_cat = np.argmax(y_test,axis=1)
    macro=f1_score(y_test_cat,y_pred, average='macro')
    pref_hist_multy_f1[i][0]=macro
    # calculate initial batch majority
    y_train_cat = np.argmax(y_test,axis=1)
    counts = np.bincount(y_train_cat)
    value = np.argmax(counts)
    majority =[value for i in range(len(y_test_cat))]
    macro=f1_score(y_test_cat,majority, average='macro')
    pref_hist_majority[i][0]=macro
    #calculate random
    y_rand=[random.randint(0,2) for i in range(len(y_test_cat))]
    macro=f1_score( y_test_cat,y_rand, average='macro')
    pref_random = pref_random + macro

    

    for index in range(1,N_QUERIES+1):

      query_index, query_instance = learner.query(X_pool)
      # print('num if query',len(query_index))
      if is_random:
        index_list = range(len(X_pool))
        query_index = random.sample(index_list,BATCH_SIZE)
      # Teach our ActiveLearner model the record it has requested.

      X, y = X_pool[query_index], y_pool[query_index]

      for j in range(5):
        learner.teach(X=X, y=y)
      # Remove the queried instance from the unlabeled pool.
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)
      print('sec',y_pool.shape)
      # Calculate and report our model's accuracy.
      model_accuracy = learner.score(data_test, test_labels)
      y_pred = learner.predict(X_test)
      
      # y_pred_cat=np.argmax(y_pred)
      y_test_cat=np.argmax(y_test,axis=1)
      # print('shapeè',y_test_cat.shape)
      macro=f1_score( y_test_cat,y_pred, average='macro')
      print('after query {n}: Accuracy :{acc:0.4f} macro f1 :{f1:0.4f}'.format(n=index + 1, acc=model_accuracy,f1=macro))
      # Save our model's performance for plotting.
      pref_hist_multy_accuracy[i][index]=model_accuracy
      pref_hist_multy_f1[i][index]=macro

       # calculate and add majority
      y_train_cat = np.argmax(y_test,axis=1)
      counts = np.bincount(y_train_cat)
      value = np.argmax(counts)
      majority =[value for i in range(len(y_test_cat))]
      macro=f1_score(y_test_cat,majority, average='macro')
      pref_hist_majority[i][index]=macro
      
    print(format_time(time.time()-t1))
  pref_random= pref_random/num_of_run
  pref_hist_majority_avg=pref_hist_majority.mean(0)
  pref_hist_multy_acc_avg=pref_hist_multy_accuracy.mean(0)
  pref_hist_multy_f1_avg=pref_hist_multy_f1.mean(0)
  return pref_hist_multy_acc_avg,pref_hist_multy_f1_avg ,pref_random,pref_hist_majority_avg#,pref_hist_multy_accuracy,pref_hist_multy_f1
    

### run_multy_committe

In [12]:
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score
from modAL.models import ActiveLearner
from modAL.batch import uncertainty_batch_sampling
from functools import partial
import random
from modAL.models import ActiveLearner, Committee

random_seed_list=[5,12,42,29,54]
commite_size = 5

def run_multy_commite(BATCH_SIZE,data_test,test_labels,num_of_run,model_name,strategy,train_size,is_random,mode):
 
  test_pool_split=0.5
  preset_batch = partial(strategy, n_instances=BATCH_SIZE)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True )
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(X_pool_0,y_pool_0, test_size=test_pool_split,shuffle=True )
  N_QUERIES = int(len(X_pool_0)/BATCH_SIZE)

  pref_hist_multy_accuracy=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_multy_f1=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_majority = np.zeros((num_of_run,N_QUERIES+1))

  pref_random = 0
  boost = False
  if mode=='boost'or mode=='bag':
    boost= True
  for i in range(num_of_run):
    clf = get_model(model_name)
    X_pool, X_train, y_pool, y_train = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True,random_state=random_seed_list[i])
    X_pool, X_test, y_pool, y_test = train_test_split(X_pool,y_pool, test_size=test_pool_split,shuffle=True,random_state=random_seed_list[i])
    learner_list = []
    for j in range(commite_size):
      learner = ActiveLearner(estimator=clf,query_strategy=preset_batch,X_training=X_train, y_training=y_train,bootstrap_init=boost,)
      learner_list.append(learner)

    committee = Committee(learner_list=learner_list)
    if mode=='bag':
      committee.rebag()
    t1 = time.time()

    # calculate initial batch
    y_pred = committee.predict(X_test)
    macro=f1_score(y_test,y_pred, average='macro')
    pref_hist_multy_f1[i][0]=macro

    # calculate initial batch majority
    counts = np.bincount(y_train)
    value = np.argmax(counts)
    majority =[value for i in range(len(y_test))]
    macro=f1_score(y_test,majority, average='macro')
    pref_hist_majority[i][0]=macro

    #calculate random
    y_rand=[random.randint(0,2) for i in range(len(y_test))]
    macro=f1_score( y_test,y_rand, average='macro')
    pref_random = pref_random + macro

    #active leraning loop
    for index in range(1,N_QUERIES+1):
      t2 = time.time()

      query_index, query_instance = committee.query(X_pool)

      if is_random : #or counter_random % random_ratio == 0:
        index_list = range(len(X_pool))
        query_index = random.sample(index_list,BATCH_SIZE)

      X, y = X_pool[query_index], y_pool[query_index]
    
      for j in range(1):
        committee.teach(X=X, y=y)

      # Remove the queried instance from the unlabeled pool.
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)

      # Calculate and report our model's accuracy.
      model_accuracy = committee.score(X_test, y_test)
      y_pred = committee.predict(X_test)

      macro=f1_score( y_test,y_pred, average='macro')
      print('after query {n}: Accuracy :{acc:0.4f} macro f1 :{f1:0.4f}'.format(n=index + 1, acc=model_accuracy,f1=macro))

      pref_hist_multy_accuracy[i][index]=model_accuracy
      pref_hist_multy_f1[i][index]=macro

      # calculate and add majority
      counts = np.bincount(y_train)
      value = np.argmax(counts)
      majority =[value for i in range(len(y_test))]
      macro=f1_score(y_test,majority, average='macro')
      pref_hist_majority[i][index]=macro
      print(format_time(time.time()-t2))

    print(format_time(time.time()-t1))
  pref_random= pref_random/num_of_run
  pref_hist_majority_avg=pref_hist_majority.mean(0)
  pref_hist_multy_acc_avg=pref_hist_multy_accuracy.mean(0)
  pref_hist_multy_f1_avg=pref_hist_multy_f1.mean(0)

  return pref_hist_multy_accuracy,pref_hist_multy_f1,pref_random,pref_hist_majority,N_QUERIES

### Run_multy_cross

In [13]:
def run_multy_cross(BATCH_SIZE,data_test,test_labels,num_of_run,model_name,strategy,train_size,is_random):
  test_pool_split=0.5
  preset_batch = partial(strategy, n_instances=40)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True )
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(X_pool_0,y_pool_0, test_size=test_pool_split,shuffle=True )
  N_QUERIES = int(len(X_pool_0)/BATCH_SIZE)

  pref_hist_multy_accuracy=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_multy_f1=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_majority = np.zeros((num_of_run,N_QUERIES+1))
  pref_random = 0
  for i in range(num_of_run):
    clf = get_model(model_name)
    X_pool, X_train, y_pool, y_train = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True,random_state=random_seed_list[i])
    X_pool, X_test, y_pool, y_test = train_test_split(X_pool,y_pool, test_size=test_pool_split,shuffle=True,random_state=random_seed_list[i])
    
    learner = ActiveLearner(
    estimator=clf,
    query_strategy=preset_batch,
    X_training=X_train, y_training=y_train)
    t1 = time.time()


    #Allow our model to query our unlabeled dataset for the most informative points according to our query strategy (uncertainty sampling)
    counter_random= 1 

    #Calculate initial batch
    y_pred = learner.predict(X_test)
    macro=f1_score(y_test,y_pred, average='macro')
    pref_hist_multy_f1[i][0]=macro

    #Calculate initial batch majority
    counts = np.bincount(y_train)
    value = np.argmax(counts)
    majority =[value for i in range(len(y_test))]
    macro=f1_score(y_test,majority, average='macro')
    pref_hist_majority[i][0]=macro

    #Calculate random
    y_rand=[random.randint(0,2) for i in range(len(y_test))]
    macro=f1_score( y_test,y_rand, average='macro')
    pref_random = pref_random + macro

    #Active Leraning loop
    for index in range(1,N_QUERIES+1):
      counter_random += 1 
      t2 = time.time()
      print(i,'th run and query number',index)
      query_index, query_instance = learner.query(X_pool)
      print('num if query',len(query_index))
      if is_random:
        index_list = range(len(X_pool))
        query_index = random.sample(index_list,BATCH_SIZE)

      X, y = X_pool[query_index], y_pool[query_index]
      for j in range(1):
        learner.teach(X=X, y=y)

      #Remove the queried instance from the unlabeled pool.
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)
      index_list = range(len(X_pool))
      query_index = random.sample(index_list,10)

      X, y = X_pool[query_index], y_pool[query_index]
      X_train = np.concatenate((X_train, X))
      y_train = np.concatenate((y_train, y))

      for j in range(1):
        learner.teach(X=X, y=y)
        
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)

      # Calculate and report our model's accuracy.
      model_accuracy = learner.score(X_test, y_test)
      y_pred = learner.predict(X_test)
      
      macro=f1_score( y_test,y_pred, average='macro')
      print('after query {n}: Accuracy :{acc:0.4f} macro f1 :{f1:0.4f}'.format(n=index + 1, acc=model_accuracy,f1=macro))

      # Save our model's performance for plotting.
      pref_hist_multy_accuracy[i][index]=model_accuracy
      pref_hist_multy_f1[i][index]=macro

      # calculate and add majority
      counts = np.bincount(y_train)
      value = np.argmax(counts)
      majority =[value for i in range(len(y_test))]
      macro=f1_score(y_test,majority, average='macro')
      pref_hist_majority[i][index]=macro
      print(format_time(time.time()-t2))
    print(format_time(time.time()-t1))

  pref_random= pref_random/num_of_run
  pref_hist_majority_avg=pref_hist_majority.mean(0)
  pref_hist_multy_acc_avg=pref_hist_multy_accuracy.mean(0)
  pref_hist_multy_f1_avg=pref_hist_multy_f1.mean(0)

  return pref_hist_multy_accuracy,pref_hist_multy_f1,pref_random,pref_hist_majority,N_QUERIES

###Run_multy for mlp that uses Keras

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, GlobalMaxPooling1D, Activation
from keras.layers import Embedding, LSTM
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
def create_keras_model_berts():
  model = Sequential()
  model.add(keras.Input(shape=(768,),name="source"))
  model.add(Dense(128, activation='relu'))
  # model.add(Dropout(0.3))
  # model.add(Dense(10, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_keras_model_GLOVE():
  model = Sequential()
  model.add(keras.Input(shape=(200,),name="source"))
  model.add(Dense(128, activation='relu'))
  # model.add(Dropout(0.3))
  # model.add(Dense(10, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

##Get model
getting all generated models that were tested

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def get_model(model_name):
  if model_name == "bag":
    clf = BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=100, max_samples=0.8)
  if model_name == "xgb":
    clf = XGBClassifier()
  if model_name == "gbc":
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=1, max_depth=1)
  if model_name =="Ada":
    clf = AdaBoostClassifier(n_estimators=200,learning_rate=0.01)
  if model_name =="svm":
    clf=svm.SVC(probability=True)
  if model_name =="rf":
    clf=RandomForestClassifier(max_depth=1000, random_state=0)
  if model_name =="LR":
    clf=LogisticRegression(random_state=0,class_weight='balanced')
  if model_name =="knn3":
    clf=KNeighborsClassifier(n_neighbors=3)
  if model_name =="knn5":
    clf=KNeighborsClassifier(n_neighbors=5)
  if model_name =="gp":
     clf=GaussianProcessClassifier()
  if model_name =="lda":
    clf=LinearDiscriminantAnalysis()
  if model_name =="qda":
    clf=QuadraticDiscriminantAnalysis() 
  return clf

##Dimension Reduction

### Sentence features with PCA
Dimension reduction using PCA for all features

In [16]:
from sklearn.decomposition import PCA
import pandas as pd

pca=PCA(0.75)
sentence_feature_normal=pca.fit_transform(np.load('/content/sentiment_new_approach/vectorization/normal_bert.npy'))
sentence_feature_tweet =pca.fit_transform(np.load('/content/sentiment_new_approach/vectorization/tweet_bert.npy'))
sentence_feature_GLOVE =pca.fit_transform(np.load('/content/sentiment_new_approach/vectorization/GLOVE.np'))
sentence_feature_GLOVE25=pca.fit_transform(np.load('/content/sentiment_new_approach/vectorization/GLOVE25.np'))
sentence_feature_GLOVE50=pca.fit_transform(np.load('/content/sentiment_new_approach/vectorization/GLOVE50.np'))
sentence_feature_GLOVE100=pca.fit_transform(np.load('/content/sentiment_new_approach/vectorization/GLOVE100.np'))


### Sentence features with PCA concatenated with metadata

In [17]:
s0=data[cols]
s1=pd.DataFrame(sentence_feature_normal)
s2=pd.DataFrame(sentence_feature_GLOVE)
s3=pd.DataFrame(sentence_feature_GLOVE25)
s4=pd.DataFrame(sentence_feature_GLOVE50)
s5=pd.DataFrame(sentence_feature_GLOVE100)
s6=pd.DataFrame(sentence_feature_tweet)
s1.reset_index(drop=True,inplace=True)
s0.reset_index(drop=True,inplace=True)
s3.reset_index(drop=True,inplace=True)
s2.reset_index(drop=True,inplace=True)
s1.reset_index(drop=True,inplace=True)
s5.reset_index(drop=True,inplace=True)
s6.reset_index(drop=True,inplace=True)


sentence_feature_normal_new=pd.concat([s1,s0],axis=1)
sentence_feature_tweet_new=pd.concat([s6,s0],axis=1)
sentence_feature_GLOVE_new=pd.concat([s2,s0],axis=1)
sentence_feature_GLOVE25_new=pd.concat([s3,s0],axis=1)
sentence_feature_GLOVE50_new=pd.concat([s4,s0],axis=1)
sentence_feature_GLOVE100_new=pd.concat([s5,s0],axis=1)


sentence_feature_normal_new=sentence_feature_normal_new.to_numpy()
sentence_feature_tweet_new=sentence_feature_tweet_new.to_numpy()
sentence_feature_GLOVE_new=sentence_feature_GLOVE_new.to_numpy()
sentence_feature_GLOVE25_new=sentence_feature_GLOVE25_new.to_numpy()
sentence_feature_GLOVE50_new=sentence_feature_GLOVE50_new.to_numpy()
sentence_feature_GLOVE100_new=sentence_feature_GLOVE100_new.to_numpy()

sentence_feature_GLOVE.shape

(6425, 44)

## Automated run settings 

In [18]:
reps_dict={
    "BERT_pca":sentence_feature_normal,
    "tweetBERT_pca":sentence_feature_tweet,
    "GLOVE_pca":sentence_feature_GLOVE,
    "GLOVE50_pca":sentence_feature_GLOVE25,
    "GLOVE25_pca":sentence_feature_GLOVE50,
    "GLOVE100_pca":sentence_feature_GLOVE100,
    "BERT_pca_concat":sentence_feature_normal_new,
    "tweetBERT_pca_concat":sentence_feature_tweet_new,
    "GLOVE_pca_concat":sentence_feature_GLOVE_new,
    "GLOVE50_pca_concat":sentence_feature_GLOVE25_new,
    "GLOVE25_pca_concat":sentence_feature_GLOVE50_new,
    "GLOVE100_pca_concat":sentence_feature_GLOVE100_new,
}

In [19]:
number_of_run = 5
def AL_loop(rep,model,AL_stg):
  
  sentence_feature=reps_dict[rep]
  data_test = sentence_feature[train_start:train_end+1]
  strat = uncertainty_sampling
  
  if AL_stg=="lc": stg=False

  if AL_stg=="Random": stg=True

  if AL_stg=="batch_lc":
    stg=False
    strat = uncertainty_batch_sampling

  if AL_stg=="qbc"or AL_stg=='boost' or AL_stg=='bag':
    stg=False
    print(AL_stg,'   ',rep,'    ',model)
    acc,f1,pref_random,pref_hist_majority,N_QUERIES = run_multy_commite(50,data_test, test_labels,number_of_run,model,strat,20,stg,AL_stg)
  
  if model=="mlp":
    if rep!= "GLOVE_pca_concat":
      print(model)
      acc,f1,pref_random,pref_hist_majority,N_QUERIES = run_multy(50, data_test, test_labels,number_of_run,create_keras_model_berts,strat,20,stg)
    else:
      acc,f1,pref_random,pref_hist_majority,N_QUERIES = run_multy(50, data_test, test_labels,number_of_run,create_keras_model_GLOVE,strat,20,stg)  
  elif AL_stg=="epsilon_in_batch":
    acc,f1,pref_random,pref_hist_majority,N_QUERIES = run_multy_epsilon_greedy(50,data_test, test_labels,number_of_run,model,strat,20,False)
  elif AL_stg=="epsilon_in_batch_rand":
    acc,f1,pref_random,pref_hist_majority,N_QUERIES = run_multy_epsilon_greedy(50,data_test, test_labels,number_of_run,model,strat,20,True)
  elif AL_stg=="cross":
    acc,f1,pref_random,pref_hist_majority,N_QUERIES = run_multy_cross(50,data_test, test_labels,number_of_run,model,strat,20,False)
  elif AL_stg!="qbc":
    acc,f1,pref_random,pref_hist_majority,N_QUERIES = run_multy_sklearn(50,data_test, test_labels,number_of_run,model,strat,20,stg)

  return acc,f1,pref_random,pref_hist_majority,N_QUERIES 
  

In [20]:
# from google.colab import files

# counter = 0
# reps = ['GLOVE']
# models = ['LR']
# stratefies = ['lc'] 

# for rep in reps:
#   for model in models:
#     for AL_stg in stratefies:
#       counter = counter+1
#       acc,f1,pref_random,pref_hist_majority,N_QUERIES = AL_loop(rep,model,AL_stg)
 

##Final run with different AL settings with Automated run setting set in previous section

In [27]:

from google.colab import files

counter = 0
reps = ['tweetBERT_pca_concat']
models = ['mlp']
stratefies = ['lc'] 

for rep in reps:
  for model in models:
    for AL_stg in stratefies:
      counter = counter+1
      acc,f1,pref_random,pref_hist_majority,N_QUERIES = AL_loop(rep,model,AL_stg)

mlp
5 20
x pool length (20, 64)
y pool length (20, 2)


ValueError: ignored

In [ ]:
acc_25perc_avg=acc[:,:int(0.25*N_QUERIES)].mean(0)
f1_25perc_avg=f1[:,:int(0.25*N_QUERIES)].mean(0)

# print(acc25_avg[-1])
print(f1_25perc_avg[-1])  


0.7591833341246902


## Word embedding
Word embedding generation in case we don't want to used the pretrained saved models that was trained previously and saved on Github.
The save representations on Github were made by code below.

### selecting test train eval set

#### 0's topic test and rest train

In [ ]:
data_cp =data

In [ ]:
topic_num = 1

In [ ]:
test_df = data[data['topic']==topic_num]
train_df = data[data['topic']!=topic_num]
train_df.head()

In [ ]:
data2=data.reset_index()
start_test=data2[data2['topic']==topic_num].iloc[0]
# end_test=data2[data2['topic']==0].iloc[-1]
# start_train =data2[data2['topic']==1].iloc[0]
# end_train = start_test=data2[data2['topic']==8].iloc[-1]
def pd_iter_func(df,topic):
    for row in df.itertuples():
        # Define your criteria here
        if row.topic ==topic:
            return row
start_test=pd_iter_func(data2,1).Index
end_test = pd_iter_func(data2,2).Index-1
train_start = pd_iter_func(data2,topic_num).Index
train_end = len(data2)
print(start_test,',',end_test)


2079 , 2092


In [ ]:
train_sentences = train_df.text.values
test_sentences = test_df.text.values
train_labels = train_df.tag.values
test_labels = test_df.tag.values
len(train_labels)

4346

#####over ride just 0 test and train for now


In [ ]:
test_labels = test_df.tag.values
train_start=0
train_end=2078

### Representation

#### BERT

In [ ]:
sentence_feature=np.load('/content/sentiment_new_approach/vectorization/normal_bert.npy')

In [ ]:
sentence_feature=np.load('/content/sentiment_new_approach/vectorization/tweet_bert.npy')

In [ ]:
sentence_feature.shape

(6425, 768)

In [ ]:
!pip install emoji


In [ ]:
from transformers import AutoModel, AutoTokenizer 
# Load BERT TWEET tokenizer and model
bert_tweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer_tweet = AutoTokenizer.from_pretrained("vinai/bertweet-base",normalization=True)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
def tokenizer_func(tokenizer_kind,sentences,labels):
  '''
  inputs:
    tokenizer_kind: is the the tokenizer of choice (normal bert, tweet bert)  
    sentences: train , dev, test
  outputs:
  torchs of 
    ids
    attention_mask
    labels
  '''
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer_kind.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 128,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                          truncation=True,
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)
  return input_ids, attention_masks ,labels
  # Print sentence 0, now as a list of IDs.


In [ ]:
sentences = data.text.values
train_labels = data.tag.values
input_ids,attention_masks,labels=tokenizer_func(tokenizer_tweet,sentences,train_labels)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
batch_size=128
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
bert_tweet.cuda()
bert_tweet.eval()
# Tracking variables 
sentence_feature=[]
t1 = time.time()
for batch in prediction_dataloader:
  t0 = time.time()  
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
      outputs = bert_tweet(b_input_ids, attention_mask=b_input_mask)
      
  sentence_features_slice = outputs[0][:,0,:].cpu().numpy()  
  # Store predictions and true labels
  sentence_feature.append(sentence_features_slice)
  elapsed = format_time(time.time() - t0)
  print("time elapse:",elapsed)
print("full time",format_time(time.time()-t1))
sentence_feature = np.concatenate(sentence_feature, axis=0)
print(sentence_feature.shape)

In [ ]:

with open('tweet_bert.npy', 'wb') as f:
    np.save(f, sentence_feature)

#### Glove word embeding

##### preprocessing

In [ ]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords
import re
import os

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Change "@name" to "Username"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    - change urls to "Link"
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Change "@name" to "Username
    s = re.sub(r'(@.*?)[\s]', 'username', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    # change urls to "Link"
    s = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'link',s)
    
    return s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import numpy as np
sentences = np.array([text_preprocessing(text) for text in data.text.values  ])



#####Glove



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/My Drive/glove.twitter.27B.zip"

Archive:  /content/gdrive/My Drive/glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [ ]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel
glove = loadGloveModel('glove.twitter.27B.50d.txt')

Loading Glove Model
1193514  words loaded!


In [ ]:

data_train = np.zeros(shape=(len(sentences),50),dtype=np.float)

for i in range(len(sentences)):
  tweet = sentences[i]
  tweet = tweet.split(' ')
  glove_word_count = 1
  for word in tweet:
    if word.lower() in glove:
      glove_word_count +=1
      data_train[i,:]+= glove[word]
  data_train[i,:]=data_train[i,:]/(glove_word_count)
features=data_train
#representation Glove

In [ ]:
sentence_feature_GLOVE=features

In [ ]:
sentence_feature_GLOVE.shape

(6425, 100)

In [ ]:
save_reesult(sentence_feature_GLOVE,'GLOVE50.np')